In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import re
pd.set_option('display.max_columns', 500)

## Scrap Function

In [ ]:
def ScrapMatch(url):

    # Faites une requête GET pour obtenir le contenu de la page
    response = requests.get(url)
    html_content = response.content

    # Utilisez Beautiful Soup pour analyser le contenu HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Configuration des sélecteurs des statistiques générales du match
    selectors = [
        {"id": "Home_Team", "selector": ".scorebox div:nth-of-type(1) div strong a"}, # Team Home
        {"id": "Away_Team", "selector": ".scorebox div:nth-of-type(2) strong a"}, # Team Away
        {"id": "Competition", "selector": ".box > div:nth-of-type(1)"}, # Competition
        {"id": "Home_Season_History", "selector": ".scorebox div:nth-of-type(1) div:nth-of-type(3)"}, # Season History Home
        {"id": "Away_Season_History", "selector": ".scorebox div:nth-of-type(2) div:nth-of-type(3)"}, # Season History Away
        {"id": "Home_Goals", "selector": "div:nth-of-type(1) div.score"}, # Goals Home
        {"id": "Away_Goals", "selector": ".scorebox div:nth-of-type(2) .scores div.score"}, # Goals Away
        {"id": "Home_xG", "selector": "div:nth-of-type(1) div.score_xg"}, # xG Home
        {"id": "Away_xG", "selector": ".scorebox div:nth-of-type(2) .scores div.score_xg"}, # xG Away
        {"id": "Home_Penalties", "selector": "div:nth-of-type(1) div.score_pen"}, # Penalties Home
        {"id": "Away_Penalties", "selector": ".scorebox div:nth-of-type(2) .scores div.score_pen"}, # Penalties Away
        {"id": "Home_Manager", "selector": ".scorebox > div:nth-of-type(1)"}, # Manager Home
        {"id": "Away_Manager", "selector": ".scorebox > div:nth-of-type(2)"}, # Manager Away
        {"id": "Home_Captain", "selector": ".scorebox > div:nth-of-type(1)"}, # Captain Home
        {"id": "Away_Captain", "selector": ".scorebox > div:nth-of-type(2)"}, # Captain Away
        {"id": "Date", "selector": ".scorebox_meta strong a"}, # Date
        {"id": "Time", "selector": "span.venuetime"}, # Time
        {"id": "Attendance-Venue-Officials", "selector": ".scorebox_meta div:nth-of-type(n+5)"}, # Affluence / Stade / Arbitres
        {"id" : "Home_Lineup", "selector" : "div#a.lineup"}, # Compo Home
        {"id" : "Away_Lineup", "selector" : "div#b.lineup"}, # Compo Away
        {"id" : "General_Statistics", "selector" : "#team_stats_extra div:nth-of-type(n+4)"}, # Stats générale
        {"id" : "General_Statistics_2", "selector" : "#team_stats tr:nth-of-type(n+2) th, td > div > div:nth-of-type(1)"}, # Stats générale 2
        {"id" : "Home_Events", "selector" : "div.a:nth-of-type(n+3) > div"}, # Home résumé
        {"id" : "Away_Events", "selector" : "div.b:nth-of-type(n+3) > div"}, # Away résumé
        {"id" : "Home_Events_Penalties", "selector" : "div.a:nth-of-type(n+34)"}, # Home tirs aux buts
        {"id" : "Away_Events_Penalties", "selector" : "div.b:nth-of-type(n+33)"} # Away tirs aux buts
    ]

    # Configuration des sélecteurs des tables statistiques des équipes
    selectors_table =[
        {"id": "Tables_Stats_Summary", "selector": "table[id*=summary]"}, # Table résumé
        {"id": "Tables_Stats_Passing", "selector": "table[id*=passing]"}, # Table Passes
        {"id": "Tables_Stats_Passing_Types", "selector": "table[id*=passing_types]"}, # Tables Type de passe
        {"id": "Tables_Stats_Defense", "selector": "table[id*=defense]"}, # Table Actions défensive
        {"id": "Tables_Stats_Possession", "selector": "table[id*=possession]"}, # Table possession
        {"id": "Tables_Stats_Misc", "selector": "table[id*=misc]"}, # Table Statistiques diverse
        {"id": "Tables_Stats_Keeper", "selector": "table[id*=keeper]"}, # Table Goals
        {"id": "Tables_Stats_Shot", "selector": "table[id*=shots_all]"} # Table Tirs
    ]

    # Récupération des statistiques générales du match avec les sélecteurs
    data_dict = {}
    for selector_info in selectors:
        element = soup.select(selector_info["selector"])
        if element:
            if len(element) == 1:
                data_dict[selector_info["id"]] = element[0].get_text()
            else:
                data_dict[selector_info["id"]] = [item.get_text() for item in element]

    # Récupération des tables avec les sélecteurs
    data_table = {}
    for selector_info in selectors_table:
        elements = soup.select(selector_info["selector"])  
        if elements:
            i=0
            for element in elements:
                table_df = pd.read_html(str(element))[0]
                if selector_info["id"] != "Tables_Stats_Shot" :
                    if i == 0 :
                        data_table["Home_"+selector_info["id"]] = table_df
                    else:
                        data_table["Away_"+selector_info["id"]] = table_df
                else:
                        data_table[selector_info["id"]] = table_df
                i+=1

    # Renommage des colonnes des dataframes
    for el in data_table:
        data_table[el].columns = data_table[el].columns.map('_'.join)
        data_table[el] = data_table[el].reset_index(drop=True)
        table_headers = data_table[el].columns.tolist()
        new_headers = [element.split("Unnamed")[1].split("_", 3)[-1] if "Unnamed" in element else element for element in table_headers]
        data_table[el].columns = new_headers

    # Séparation des tables en vu du merge
    Home_Players_Stats = [value for key, value in data_table.items() if "Home" in key and "Keeper" not in key]
    Away_Players_Stats = [value for key, value in data_table.items() if "Away" in key and "Keeper" not in key]
    Home_Goalkeeper_Stats = [value for key, value in data_table.items() if "Home_Tables_Stats_Keeper" in key]
    Away_Goalkeeper_Stats = [value for key, value in data_table.items() if "Away_Tables_Stats_Keeper" in key]
    Shots_Stats = [value for key, value in data_table.items() if "Shot" in key]
    if Home_Goalkeeper_Stats: Home_Goalkeeper_Stats = Home_Goalkeeper_Stats[0]
    if Away_Goalkeeper_Stats: Away_Goalkeeper_Stats = Away_Goalkeeper_Stats[0]
    if Shots_Stats: Shots_Stats = Shots_Stats[0]

    # Merge pour la liste data_table_home
    if Home_Players_Stats:
        df = Home_Players_Stats[0]
        for i in range(1, len(Home_Players_Stats)):
            common_cols = set(df.columns).intersection(Home_Players_Stats[i].columns)
            df = df.merge(Home_Players_Stats[i], how='inner', left_on=list(common_cols), right_on=list(common_cols))
        Home_Players_Stats = df.copy()

    # Merge pour la liste data_table_away
    if Away_Players_Stats:
        df = Away_Players_Stats[0]
        for i in range(1, len(Away_Players_Stats)):
            common_cols = set(df.columns).intersection(Away_Players_Stats[i].columns)
            df = df.merge(Away_Players_Stats[i], how='inner', left_on=list(common_cols), right_on=list(common_cols))
        Away_Players_Stats = df.copy()

    # Transformation des dataframes statistiques en dictionnaire et ajout au dictionnaire final
    lst_df = [Home_Players_Stats, Away_Players_Stats, Home_Goalkeeper_Stats, Away_Goalkeeper_Stats, Shots_Stats]

    for df in lst_df:
        result_dict = {}

        if isinstance(df, pd.DataFrame):
            for index, row in df.iterrows():
                key = row['Player']
                values_dict = row.drop('Player').to_dict()
                result_dict[key] = values_dict

            df_name = [name for name, var in locals().items() if var is df][0]

            if df_name in data_dict:
                data_dict[df_name].update(result_dict)
            else:
                data_dict[df_name] = result_dict

    return data_dict

## Cleaning function

In [ ]:
def CleanMatch(match):

    clean_match = {} # Our new dictionnary with cleaned data

    # ------- Teams ------- #
    clean_match['Teams'] = {'Home':match['Home_Team'],'Away':match['Away_Team']}

    # ------- Competition ------- #
    competition = re.split(r'\s+\(', match['Competition'])
    competition = [partie.strip('()') for partie in competition]
    match['Competition'] = competition[0]
    match['Matchweek'] = competition[1]
    if 'Matchweek' in match['Matchweek']:
        match['Matchweek'] = int(match['Matchweek'].replace('Matchweek ', ''))
    else:
        match['Matchweek'] = match['Matchweek'].split(")")[0]
    clean_match['Competition'] = match['Competition']
    clean_match['Matchweek'] = match['Matchweek']

    # ------- Season History ------- #
    for team_history in ['Home_Season_History', 'Away_Season_History']:
        if team_history in match:
            if isinstance(match[team_history], list):
                match.pop(team_history)
            elif 'Match' in match[team_history]:
                match.pop(team_history)
            else:
                dict_history = {}
                match[team_history] = match[team_history].split('-')
                dict_history = {'Victory':match[team_history][0],
                                'Draw':match[team_history][1],
                                'Defeat':match[team_history][2]}
                match[team_history] = dict_history
                if 'Season_History' not in clean_match:
                    clean_match['Season_History'] = {}
                if team_history == 'Home_Season_History':
                    clean_match['Season_History']['Home'] = match[team_history]
                else:
                    clean_match['Season_History']['Away'] = match[team_history]

    # ------- Goals ------- #
    if len(match['Home_Goals']) > 1 : match['Home_Goals'] = int(match['Home_Goals'][0]) 
    match['Away_Goals'] = int(match['Away_Goals'])
    clean_match['Goals'] = {'Home':match['Home_Goals'],
                           'Away':match['Away_Goals']}
    
    # -------- Penalties -------- #
    if 'Home_Penalties' in match and 'Away_Penalties' in match:
        if len(match['Home_Penalties']) > 1: match['Home_Penalties'] = int(match['Home_Penalties'][0])
        match['Away_Penalties'] = int(match['Away_Penalties'])
        clean_match['Penalties'] = {'Home':match['Home_Penalties'],
                                    'Away':match['Away_Penalties']}


    # -------- xG -------- #
    if 'Home_xG' in match and 'Away_xG' in match:
        if len(match['Home_xG']) > 1 : match['Home_xG'] = float(match['Home_xG'][0].replace(",", "."))
        match['Away_xG'] = float((match['Away_xG']).replace(',', '.'))
        clean_match['xG'] = {'Home':match['Home_xG'],
                            'Away':match['Away_xG']}
    
    # -------- Manager -------- #
    lst = ['Home_Manager', 'Away_Manager']
    for el_lst in lst:
        if el_lst in match:
            match[el_lst] = match[el_lst].split('\n')
            for el in match[el_lst]:
                if 'Manager' in el:
                    el = el.replace('\xa0', ' ').replace('Manager: ', '')
                    match[el_lst] = el
    clean_match['Manager'] = {'Home':match['Home_Manager'],
                        'Away':match['Away_Manager']}
    
    # -------- Captain -------- #
    lst = ['Home_Captain', 'Away_Captain']
    for el_lst in lst:
        if el_lst in match:
            match[el_lst] = match[el_lst].split('\n')
            for el in match[el_lst]:
                if 'Captain' in el:
                    el = el.replace('\xa0', ' ').replace('Captain: ', '')
                    match[el_lst] = el
    clean_match['Captain'] = {'Home':match['Home_Captain'],
                    'Away':match['Away_Captain']}
    
    # -------- Time -------- #
    if 'Time' in match:
        match['Time'] = match['Time'].replace(' (venue time)', '')
        clean_match['Time'] = match['Time']

    # -------- Attendance / Stade / Arbitres -------- #
    if match['Attendance-Venue-Officials']:
        for el in match['Attendance-Venue-Officials']:
            if 'Attendance' in el:
                match['Attendance'] = int(el.replace('Attendance: ', '').replace(',',''))
                clean_match['Attendance'] = match['Attendance']
            if 'Venue' in el:
                match['Venue'] = el.replace('Venue: ', '')
                clean_match['Venue'] = match['Venue']
            if 'Officials' in el:
                    officials = el.replace('Officials: ', '')
                    if officials != '' :
                        officials = officials.split('\xa0· ')
                        match['Officials'] = [element.replace('\xa0', ' ') for element in officials]
                        dict_arbitres = {}
                        for official in match['Officials'] :
                            official = official.split(" (")
                            official[1] = official[1].replace(")", "")
                            dict_arbitres[official[1]] = official[0]
                        match['Officials'] = dict_arbitres
                        clean_match['Officials'] = match['Officials']

    # -------- Lineup & Formation -------- #
    if 'Home_Lineup' in match and 'Away_Lineup' in match:
        lst_lineup = ['Home_Lineup', 'Away_Lineup']
        for el in lst_lineup:
            lineup = match[el].split('\n')
            lineup = [el for el in lineup if el != '']
            formation = re.split(r'\s+\(', lineup[0])[1].replace(')','')
            starting_list = re.findall(r'(\d+)([^\d]+)', lineup[1])
            substitute_list = re.findall(r'(\d+)([^\d]+)', lineup[3])
            starting = {}
            substitute = {}
            for num, name in starting_list: starting[num] = name
            for num, name in substitute_list: substitute[num] = name
            lineup = {
                'Formation' : formation,
                'Starting': starting,
                'Substitute': substitute
            }
            match[el] = lineup
        clean_match['Lineup'] = {'Home':match['Home_Lineup'],
                                 'Away':match['Away_Lineup']}
        

    # -------- General Statistics -------- #
    if 'General_Statistics' in match:
        sublists = []
        for i in range(0, len(match['General_Statistics']), 3):
            sublist = match['General_Statistics'][i:i+3]
            sublists.append(sublist) 
        for lst in sublists :
            clean_match[lst[1]] = {'Home':int(lst[0]),
                                   'Away':int(lst[2])}

    # -------- General Statistics 2 -------- #
    if 'General_Statistics_2' in match:
        sublists = []
        general_stats_2 = {}
        for i in range(0, len(match['General_Statistics_2']), 3):
            sublist = match['General_Statistics_2'][i:i+3]
            if "Cards" not in sublist:
                sublists.append(sublist) 
        for lst in sublists :
            general_stats_2[lst[0]] = {'Home':lst[1], 'Away':lst[2]}
        for key, value in general_stats_2.items():
            teams = ['Home', 'Away']
            for team in teams:
                if "\xa0" in value[team]:
                    new_value = value[team].replace("\xa0", " ")
                    new_value = new_value.replace("%", "")
                    new_value = new_value.split(" — ")
                    general_stats_2[key][team] = {'Success':'', 'Failed':'', 'Percentage':''}
                    for el in new_value:
                        if "of" in el:
                            el = el.split(" of ")
                            general_stats_2[key][team]['Success'] = int(el[0])
                            general_stats_2[key][team]['Failed'] = int(el[1])
                        else:
                            if el == '':
                                general_stats_2[key][team]['Percentage'] = 0
                            else :
                                general_stats_2[key][team]['Percentage'] = int(el)
                else:
                    new_value = int(value[team].replace("%", ""))
                    general_stats_2[key][team] = new_value
        clean_match.update(general_stats_2)

    # -------- Events -------- #
    if 'Home_Events' in match and 'Away_Events' in match:
        clean_match['Events'] = {}
        dict_events = {}
        for events in ['Home_Events', 'Away_Events'] :
            list_events = match[events]
            list_events = [event.replace('\n', '').replace('\t', '').replace('\xa0', '') for event in list_events]
            sublists = [list_events[i:i+2] for i in range(0, len(list_events), 2)]
            for el in sublists:
                if "’" in el[0]:

                    temp_events = {}
                    type_team = clean_match['Teams']['Home'] if events == 'Home_Events' else clean_match['Teams']['Away']
                    minute = el[0].split('’')[0]
                    score = el[0].split('’')[1]

                    if "Own Goal" in el[1]:
                        player = el[1].split("Own Goal")[0]
                        temp_events = {'Team': type_team,'Event':'Own Goal','Player':player,'Score':score}
                        if minute in dict_events:
                            if not isinstance(dict_events[minute], list):
                                dict_events[minute] = [dict_events[minute]]
                            dict_events[minute].append(temp_events)
                        else:
                            dict_events[minute] = temp_events
                        continue
                        
                    if "Penalty saved" in el[1]:
                        player = el[1].split("Penalty saved by ")[0]
                        player_saved_by = el[1].split("Penalty saved by ")[1].replace('—', '')
                        temp_events = {'Team': type_team,'Event':'Penalty Saved','Player':player,'Saved By':player_saved_by,'Score':score}
                        if minute in dict_events:
                            if not isinstance(dict_events[minute], list):
                                dict_events[minute] = [dict_events[minute]]
                            dict_events[minute].append(temp_events)
                        else:
                            dict_events[minute] = temp_events
                        continue

                    if "Penalty Miss" in el[1]:
                        player = el[1].split('—')[0].replace('Penalty Miss', '').split("—")[0]
                        temp_events = {'Team': type_team,'Event':'Penalty Miss','Player':player,'Score':score}
                        if minute in dict_events:
                            if not isinstance(dict_events[minute], list):
                                dict_events[minute] = [dict_events[minute]]
                            dict_events[minute].append(temp_events)
                        else:
                            dict_events[minute] = temp_events
                        continue
                        
                    if "Penalty Kick" in el[1]:
                        scorer = el[1].split('—')[0].replace('Penalty Kick', '')
                        temp_events = {'Team': type_team,'Event' : "Goal (Penalty Kick)",'Score':score,'Scorer':scorer}
                        if minute in dict_events:
                            if not isinstance(dict_events[minute], list):
                                dict_events[minute] = [dict_events[minute]]
                            dict_events[minute].append(temp_events)
                        else:
                            dict_events[minute] = temp_events
                        continue
               
                    if "Goal" in el[1]:
                        scorer = el[1].split('—')[0]
                        if "Assist:" in scorer :
                            assister = scorer.split('Assist:')[1]
                            scorer = scorer.split('Assist:')[0]
                            temp_events = {'Team':type_team,'Event':"Goal",'Score':score,'Scorer':scorer,'Assister':assister}
                            if minute in dict_events:
                                if not isinstance(dict_events[minute], list):
                                    dict_events[minute] = [dict_events[minute]]
                                dict_events[minute].append(temp_events)
                            else:
                                dict_events[minute] = temp_events
                            continue
                        else:
                            temp_events = {'Team': type_team,'Event':"Goal",'Score':score,'Scorer':scorer}
                            if minute in dict_events:
                                if not isinstance(dict_events[minute], list):
                                    dict_events[minute] = [dict_events[minute]]
                                dict_events[minute].append(temp_events)
                            else:
                                dict_events[minute] = temp_events
                            continue
                        
                    if "for " in el[1] and "Substitute" in el[1]:
                        incoming = el[1].split('for ')[0]
                        outcoming = el[1].split('for ')[1].split('—')[0]
                        temp_events = {'Team': type_team,'Event' : "Substitute",'Score':score,'Incoming':incoming,'Outcoming':outcoming}
                        if minute in dict_events:
                            if not isinstance(dict_events[minute], list):
                                dict_events[minute] = [dict_events[minute]]
                            dict_events[minute].append(temp_events)
                        else:
                            dict_events[minute] = temp_events
                        continue
                        
                    if "Yellow Card" in el[1] or "Red Card" in el[1]:
                        player = el[1].split('—')[0]
                        type_card = el[1].split('—')[1]
                        temp_events = {'Team':type_team,'Event':type_card,'Score':score,'Player':player}
                        if minute in dict_events:
                            if not isinstance(dict_events[minute], list):
                                dict_events[minute] = [dict_events[minute]]
                            dict_events[minute].append(temp_events)
                        else:
                            dict_events[minute] = temp_events
                        continue

            # Update final dict 
            clean_match['Events'].update(dict_events)

        # Sorting events by minute and conserved the type of key (string)
        tuple_list = [(int(key.split('+')[0]), int(key.split('+')[1]) if '+' in key else 0, key, value) for key, value in clean_match['Events'].items()]
        sorted_tuple_list = sorted(tuple_list, key=lambda x: (x[0], x[1]))
        sorted_data = {key: value for _, _, key, value in sorted_tuple_list}
        clean_match['Events'] = {key: sorted_data[key] for _, _, key, _ in sorted_tuple_list}
    
    # -------- Penaltys -------- #
    teams = ['Home_Events_Penalties', 'Away_Events_Penalties']
    if 'Home_Events_Penalties' in match and 'Away_Events_Penalties' in match:
        clean_match['Penalties Events'] = {}
        for team in teams:
            if team in match :
                is_first_iteration = True 
                score_precedent = ""
                for el in match[team]:
                    el = el.replace('\t','').split('\n\n\n\n\n\n')
                    el[0] = el[0].replace('\n\n\n\xa0', '').split('\n')    
                    el[1] = el[1].split('\n\n\n\xa0')
                    num_penaltie = int(el[0][0])
                    score = el[0][1]
                    player = el[1][0]
                    if is_first_iteration:
                        if team == 'Home_Events_Penalties':
                            result = 'Failed' if score.split(':')[0] == '0' else 'Successed'
                        else:
                            result = 'Failed' if score.split(':')[1] == '0' else 'Successed'
                        is_first_iteration = False 
                        score_precedent = score
                    else:
                        if team == 'Home_Events_Penalties':
                            result = 'Failed' if score_precedent.split(':')[0] == score.split(':')[0] else 'Successed'
                            score_precedent = score
                        else:
                            result = 'Failed' if score_precedent.split(':')[1] == score.split(':')[1] else 'Successed'
                            score_precedent = score
                    clean_match['Penalties Events'][num_penaltie] = {'Result':result,'Score':score,'Player':player}

    # -------- Players statistics -------- #
    players_stats = ['Home_Players_Stats', 'Away_Players_Stats']
    clean_match['Players Statistics'] = {}
    for stats in players_stats :
        if stats in match:
            if stats == 'Home_Players_Stats':
                clean_match['Players Statistics']['Home'] = match['Home_Players_Stats']
            else:
                clean_match['Players Statistics']['Away'] = match['Away_Players_Stats']

    # -------- Goalkeeper statistics -------- #
    goals_stats = ['Home_Goalkeeper_Stats', 'Away_Goalkeeper_Stats']
    clean_match['Goalkeepers Statistics'] = {}
    for stats in goals_stats :
        if stats in match:
            if stats == 'Home_Goalkeeper_Stats':
                clean_match['Goalkeepers Statistics']['Home'] = match['Home_Goalkeeper_Stats']
            else:
                clean_match['Goalkeepers Statistics']['Away'] = match['Away_Goalkeeper_Stats']


    return clean_match

# Test

In [ ]:
#url = "https://fbref.com/en/matchs/8689fb39/Celta-Vigo-Osasuna-13-Aout-2023-La-Liga"
#url = "https://fbref.com/en/matchs/d63657b1/Fulham-Norwich-City-18-Aout-2012-Premier-League"
#url = "https://fbref.com/en/matchs/0192aa5d/Al-Hazem-Al-Ahli-19-Aout-2021-Saudi-Professional-League"
url = "https://fbref.com/en/matchs/7140acae/Argentina-France-18-Decembre-2022-FIFA-World-Cup"
#url = "https://fbref.com/en/matchs/85c72a0a/France-Morocco-14-Decembre-2022-FIFA-World-Cup"
#url = "https://fbref.com/en/matchs/f4fb0301/Liverpool-West-Ham-United-26-Aout-2006-Premier-League"
#url = "https://fbref.com/en/matchs/c1e42359/Almeria-Real-Madrid-14-Aout-2022-La-Liga"
#url = "https://fbref.com/en/matchs/6e2ac192/Norwich-City-Queens-Park-Rangers-25-Aout-2012-Premier-League"
#url = "https://fbref.com/en/matchs/87fcc186/Paris-FC-US-Concarneau-26-Aout-2023-Ligue-2"
#url = "https://fbref.com/en/matchs/9b5a5b22/Manchester-City-Bayern-Munich-11-Avril-2023-Champions-League"
#url = "https://fbref.com/en/matchs/3b8c18de/Bayern-Munich-Manchester-City-19-Avril-2023-Champions-League"
#url = "https://fbref.com/en/matchs/1f3e596b/Annecy-Toulouse-6-Avril-2023-Coupe-de-France"
url = "https://fbref.com/en/matchs/33cc6a3a/Greuther-Furth-Paderborn-07-30-Juillet-2023-2-Bundesliga"
#url = "https://fbref.com/en/matches/74e4b97c/Monaco-Nice-September-22-2023-Ligue-1" # Penalty arrété 
#url = "https://fbref.com/en/matchs/f64f7da2/Uruguay-Ghana-2-Juillet-2010-FIFA-World-Cup" # Penalty manqué
#url = "https://fbref.com/en/matchs/f2fb61f7/Le-Havre-Lille-1-Octobre-2023-Ligue-1" # Own goal

match = ScrapMatch(url)
t = CleanMatch(match)